<a href="https://colab.research.google.com/github/prajualpillai/Battle-of-Neighborhoods/blob/master/Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Battle of Neighborhoods

## Discription of problem:

The problem we are dealing with is that if we were to open up a store in an area one would have to decide upon a number of factors.
One of those factors is the local competition. If one was to open up a store for example a food store in an area where there are already 20+ stores present then the opening up of a new store might not have a great impact. There is another way to look at this which is that from the size of current market you can judge that how big a particular industry is in that market. For exapmle if you have a different idea or approch for your idea you can reap larger profits if you enter a region where food interests a larger number of people.

## Discription of data:
I will be using the new york data provided by Capstone in an a=earlier assignment and the toronto data obtained from foursquare in order reach upon a particular conclusion.

## Methodology:
I will use the foursqaure database in order to find the areas of interest in Manhattan and Toronto and after obtaining those areas of interest I will be performing cluster analysis in order to segment the areas of interest.
After the clustering we can study in detail about the interests and market size for indiviadual cities and are thier any markets which are absent in one and which could be exploited.
Is any market already saturated such that exploiting that any further might not give out a reasonable profit.

In [0]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup as bs

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
torronto = req.get(url)

In [3]:
torronto.status_code

200

In [0]:
# Extracting content from the given data
torronto_data = torronto.content

In [0]:
# Converting to lxml form using BeautifulSoup
torronto_data = bs(torronto_data, "lxml")

In [0]:
#taking out the data we need
def func(soup):
  links = soup.find_all("td")
  i = []
  nn = []
  nn1=[]
  c=0
  for l in links:
      l = str(l)
      if len(l) >= 500: #we know the biggest sentance/address is around 150 words so we do this to avoid useless iterations
          break
      if len(l) == 13:
          i.append(l[4:-6])   # we eliminate the <td> and </td> present at the starting and ending therefore giving us the entire sentance
      else:
          if c==0:
              nn.append(l[4:-6])
              c+=1
          else:
              nn1.append(l[4:-6])
              c=0
  return(i,nn,nn1)

In [0]:
# Converting to dataframe
i,nn,nn1 = func(torronto_data)

torr = pd.DataFrame({
    "Postal Code":i,
    "Borough":nn,
    "Neighborhood":nn1
},columns = ["Postal Code","Borough","Neighborhood"])

In [0]:
# Considering only those values for which Not Assigned is not there
df2 = torr[torr["Borough"]!="Not assigned"]
df2 = df2.reset_index(drop = True)

In [9]:
# checking for null values
df2["Neighborhood"].isnull().value_counts()

False    103
Name: Neighborhood, dtype: int64

In [10]:
!pip install geocoder
!pip install geopy
import geocoder
from geopy.geocoders import Nominatim

     |████████████████████████████████| 102kB 2.7MB/s 


In [11]:
loca = pd.read_csv("Geospatial_Coordinates.csv")
loca.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
final = pd.merge(df2,loca[["Postal Code","Latitude","Longitude"]], on = 'Postal Code')
final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [0]:
final.drop(["Postal Code"],axis=1,inplace = True)

In [0]:
# Only taking in boroughs in Toronto
g = -1
col = final.columns
final1 = pd.DataFrame(columns = col)
for f in final["Borough"]:
  g+=1
  if f[-7:] == 'Toronto':
    final1 = final1.append({"Borough": final.loc[g,"Borough"],
                            "Neighborhood": final.loc[g, "Neighborhood"],
                            "Latitude": final.loc[g, "Latitude"],
                            "Longitude": final.loc[g, "Longitude"]}, ignore_index=True)

In [15]:
final1.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [58]:
set(final1["Neighborhood"])

{'Berczy Park',
 'Brockton, Parkdale Village, Exhibition Place',
 'Business reply mail Processing Centre, South Central Letter Processing Plant Toronto',
 'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport',
 'Central Bay Street',
 'Christie',
 'Church and Wellesley',
 'Commerce Court, Victoria Hotel',
 'Davisville',
 'Davisville North',
 'Dufferin, Dovercourt Village',
 'First Canadian Place, Underground city',
 'Forest Hill North &amp; West, Forest Hill Road Park',
 'Garden District, Ryerson',
 'Harbourfront East, Union Station, Toronto Islands',
 'High Park, The Junction South',
 'India Bazaar, The Beaches West',
 'Kensington Market, Chinatown, Grange Park',
 'Lawrence Park',
 'Little Portugal, Trinity',
 'Moore Park, Summerhill East',
 'North Toronto West,  Lawrence Park',
 'Parkdale, Roncesvalles',
 "Queen's Park, Ontario Provincial Government",
 'Regent Park, Harbourfront',
 'Richmond, Adelaide, King',
 'Rosedale',
 'Rosela

In [0]:
# Downloading data of new york
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [0]:
import json
from pandas import json_normalize as jn

In [0]:
with open("newyork_data.json") as x:
  data = json.load(x)

In [0]:
new_york = jn(data)

In [0]:
nn_data = new_york["features"]

In [21]:
nn_data[0][1]

{'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.2',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Co-op',
  'annoline2': 'City',
  'annoline3': None,
  'bbox': [-73.82993910812398,
   40.87429419303012,
   -73.82993910812398,
   40.87429419303012],
  'borough': 'Bronx',
  'name': 'Co-op City',
  'stacked': 2},
 'type': 'Feature'}

In [22]:
len(nn_data[0])

306

In [0]:
ny_nn = pd.DataFrame(columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'])

In [0]:
# Creating dataframe for processing
for i in range(len(nn_data[0])):
  borough = nn_data[0][i]['properties']["borough"]
  longitude,latitude = nn_data[0][i]['geometry']['coordinates']
  neighborhood = nn_data[0][i]['properties']['name']
  ny_nn = ny_nn.append({
      "Borough":borough,
      "Neighborhood":neighborhood,
      "Latitude":latitude,
      "Longitude":longitude
  },ignore_index = True)

In [25]:
ny_nn.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [26]:
len(set(ny_nn['Borough']))

5

In [27]:
# Only selecting boroughs in Manhattan
nynn = ny_nn[ny_nn["Borough"]=='Manhattan'].reset_index(drop = True)
nynn.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [60]:
set(nynn["Neighborhood"])

{'Battery Park City',
 'Carnegie Hill',
 'Central Harlem',
 'Chelsea',
 'Chinatown',
 'Civic Center',
 'Clinton',
 'East Harlem',
 'East Village',
 'Financial District',
 'Flatiron',
 'Gramercy',
 'Greenwich Village',
 'Hamilton Heights',
 'Hudson Yards',
 'Inwood',
 'Lenox Hill',
 'Lincoln Square',
 'Little Italy',
 'Lower East Side',
 'Manhattan Valley',
 'Manhattanville',
 'Marble Hill',
 'Midtown',
 'Midtown South',
 'Morningside Heights',
 'Murray Hill',
 'Noho',
 'Roosevelt Island',
 'Soho',
 'Stuyvesant Town',
 'Sutton Place',
 'Tribeca',
 'Tudor City',
 'Turtle Bay',
 'Upper East Side',
 'Upper West Side',
 'Washington Heights',
 'West Village',
 'Yorkville'}

In [0]:
# Introducing a foursquare function get_categories in order to get the categories of the venues we get
def get_category_type(row):
  try:
    cat_lis = row['categories']
  except:
    cat_lis = row['venue.categories']
  if len(cat_lis)==0:
    return None
  else:
    return cat_lis[0]['name']

### Defining function to get the data of a location that is mentioned

In [0]:
def venue_data(names, latitudes, longitudes):
  venues_list = []
  c=0

  for name, lat, lng in zip(names, latitudes, longitudes):

    c_id = 'EE500GIJJWSAWLLSAJFKXSDXZ1KHSITBDHWYW52RTH4BKKAK'
    c_se = 'QM1D2VGHTUF3IN00AYTV25PRY0WOID203BG2Q5JZPXPMADLF'
    ver = '20180605'
    radius = 500
    limit=100
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(c_id,c_se,lat,lng,ver,radius,limit)
    result = req.get(url).json()
    venue = result['response']['groups'][0]['items']
    if len(venue) > 0:
      venues_list.append([(name, lat, lng, v['venue']['name'],v['venue']['location']['lat'], v['venue']['location']['lng'],
                          v['venue']['categories'][0]['name']) for v in venue])
      
      n_ven1 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

      n_ven1.columns = ['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']

  return n_ven1

In [0]:
ven_ny = venue_data(nynn['Neighborhood'],nynn['Latitude'],nynn['Longitude'])
#nynn['Latitude']

In [0]:
ven_tr = venue_data(names=final1['Neighborhood'],
                                   latitudes=final1['Latitude'],
                                   longitudes=final1['Longitude']
                                  )

In [0]:
from sklearn.cluster import KMeans

In [34]:
lny = len(set(ven_ny['Venue Category']))
ltr = len(set(ven_tr['Venue Category']))
print("The number of categories in manhattan = ", lny)
print("The number of categories in toronto = ", ltr)

The number of categories in manhattan =  331
The number of categories in toronto =  237


### We can see that just on the basis of the type of categories manhattan has a way more diverse feature set than toronto

# Doing one hot encoding on both the datasets

In [0]:
 def oh(ven): 
  ven_oh = pd.get_dummies(ven[['Venue Category']], prefix ="", prefix_sep="")
  #print(ven_oh.head())
  #ven_oh.drop(["Neighborhood"],axis=1,inplace=True)
  ven_oh['Neighborhood'] = ven['Neighborhood']
  cols = [ven_oh.columns[-1]]+ list(ven_oh.columns[:-1])
  ven_oh = ven_oh[cols]
  gr = ven_oh.groupby('Neighborhood').mean().reset_index()
  return gr

In [0]:
ny_oh = oh(ven_ny)
tr_oh = oh(ven_tr)

In [37]:
ny_oh.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,...,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.000000,0.015385,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.015385,0.015385,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.015385,0.0,0.0,0.0,0.000000,0.015385,0.0,0.0,0.0,0.00,0.0,0.015385,0.0,0.015385,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.015385,0.0,0.00,0.000000
1,Carnegie Hill,0.0,0.0,0.0,0.000000,0.011494,0.0,0.0,0.0,0.011494,0.000000,0.011494,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.011494,0.022989,0.0,0.022989,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.034483,0.000000,0.0,...,0.0,0.022989,0.0,0.011494,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011494,0.0,0.0,0.0,0.0,0.022989,0.0,0.0,0.0,0.011494,0.034483,0.0,0.00,0.034483
2,Central Harlem,0.0,0.0,0.0,0.066667,0.044444,0.0,0.0,0.0,0.000000,0.022222,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.022222,0.0,0.022222,0.000000,0.0,0.044444,0.0,0.0,0.0,0.022222,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.022222,0.022222,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000
3,Chelsea,0.0,0.0,0.0,0.000000,0.030000,0.0,0.0,0.0,0.000000,0.070000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.010000,0.030000,0.0,0.020000,0.0,0.0,0.0,0.010000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.010000,0.010000,0.0,...,0.0,0.010000,0.0,0.010000,0.0,0.0,0.01,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.010000,0.000000,0.0,0.01,0.000000
4,Chinatown,0.0,0.0,0.0,0.000000,0.030000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.02,0.000000,0.000000,0.0,0.01,0.000000,0.0,0.000000,0.040000,0.0,0.030000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.000000,0.000000,0.020000,0.0,...,0.0,0.010000,0.0,0.000000,0.0,0.0,0.00,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.010000


In [38]:
tr_oh.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017857,0.000000,0.0,0.0,0.0000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.000000,0.0,0.000000,0.0,0.017857,0.0000,0.000000,0.0,0.000000,...,0.035714,0.0,0.017857,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.017857,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.000000,0.086957,0.0,0.043478,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.086957,0.0000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.043478,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0625,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0625,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0625,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.055556,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.055556,0.0,0.055556,0.0,0.000000,0.0000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,Central Bay Street,0.015385,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.015385,0.0,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.015385,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0000,0.030769,0.0,0.030769,...,0.015385,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.015385,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385,0.0,0.0,0.0


# Find 5 top venues in both the cities

What we will be looking for is that if we were to open a shop in any of the two cities which type of shop has the most traction among consumers in individual city and how they vary among themselves

In [0]:
def common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
def top_venues(gr,num_top_venues):

  indicators = ['st', 'nd', 'rd']

  # create columns according to number of top venues
  columns = ['Neighborhood']
  for ind in np.arange(num_top_venues):
      try:
          columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
      except:
          columns.append('{}th Most Common Venue'.format(ind+1))

  # create a new dataframe
  neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
  neighborhoods_venues_sorted['Neighborhood'] = gr['Neighborhood']

  for ind in np.arange(gr.shape[0]):
      neighborhoods_venues_sorted.iloc[ind, 1:] = common_venues(gr.iloc[ind, :], num_top_venues)

  return neighborhoods_venues_sorted


In [61]:
ny_top = top_venues(ny_oh,5)
ny_top.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Bookstore,Gym / Fitness Center
2,Central Harlem,African Restaurant,Seafood Restaurant,Gym / Fitness Center,American Restaurant,Bar
3,Chelsea,Coffee Shop,Art Gallery,Ice Cream Shop,Café,American Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Coffee Shop
5,Civic Center,Coffee Shop,Hotel,Cocktail Bar,Spa,Park
6,Clinton,Theater,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Hotel
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Sandwich Place
8,East Village,Mexican Restaurant,Bar,Coffee Shop,Cocktail Bar,Speakeasy
9,Financial District,Coffee Shop,Pizza Place,Hotel,American Restaurant,Italian Restaurant


In [62]:
tr_top = top_venues(tr_oh,5)
tr_top.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Grocery Store
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boutique,Rental Car Location
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place
5,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant
8,Davisville,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Gym
9,Davisville North,Park,Pizza Place,Breakfast Spot,Sandwich Place,Food & Drink Shop


Now we will form clusters in individual cities and then compare the

In [0]:
def clustering(gr,top,original,kclusters):
  gr_clstr = gr.drop(["Neighborhood"],axis=1)
  kmean = KMeans(n_clusters = kclusters,random_state=0).fit(gr_clstr)
  #print(kmean.labels_[0:10])
  top.insert(0,'Cluster Labels',kmean.labels_)
  merged = original
  merged = merged.join(top.set_index('Neighborhood'), on = 'Neighborhood')
  return merged

In [74]:
tr_clstr = clustering(tr_oh,tr_top,final1,5)
tr_clstr.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub
4,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Doner Restaurant,Dim Sum Restaurant


In [75]:
ny_clstr = clustering(ny_oh,ny_top,nynn,5)
ny_clstr.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Pharmacy
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Coffee Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Deli / Bodega,Mobile Phone Shop,Chinese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Lounge,Bakery,Café,Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Mexican Restaurant,Coffee Shop,Deli / Bodega,Café


# Results:

In [95]:
for i in range(0,5):
  print(f"The size of cluster {i} of Toronto is", len(tr_clstr.loc[tr_clstr["Cluster Labels"] == i]))
  print(f"The size of cluster {i} of Manhattan is", len(ny_clstr.loc[ny_clstr["Cluster Labels"] == i]))

The size of cluster 0 of Toronto is 27
The size of cluster 0 of Manhattan is 19
The size of cluster 1 of Toronto is 8
The size of cluster 1 of Manhattan is 9
The size of cluster 2 of Toronto is 2
The size of cluster 2 of Manhattan is 1
The size of cluster 3 of Toronto is 1
The size of cluster 3 of Manhattan is 10
The size of cluster 4 of Toronto is 1
The size of cluster 4 of Manhattan is 1


We see that clusters 0 are the leading clusters for both the cities but lets investigate further what these clusters have

In [92]:
clstr0_tr = tr_clstr.loc[tr_clstr["Cluster Labels"]==0].reset_index(drop=True)
clstr0_tr.head(27)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop
5,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place
6,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Candy Store,Italian Restaurant
7,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega
8,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant
9,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Asian Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Café


In [93]:
clstr0_ny = ny_clstr.loc[ny_clstr["Cluster Labels"]==0].reset_index(drop=True)
clstr0_ny.head(19)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Coffee Shop
1,Manhattan,Central Harlem,40.815976,-73.943211,0,African Restaurant,Seafood Restaurant,Gym / Fitness Center,American Restaurant,Bar
2,Manhattan,Upper East Side,40.775639,-73.960508,0,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Bakery,Yoga Studio
3,Manhattan,Yorkville,40.775930,-73.947118,0,Coffee Shop,Gym,Italian Restaurant,Bar,Sushi Restaurant
4,Manhattan,Lenox Hill,40.768113,-73.958860,0,Coffee Shop,Pizza Place,Italian Restaurant,Sushi Restaurant,Café
5,Manhattan,Upper West Side,40.787658,-73.977059,0,Bar,Italian Restaurant,Dessert Shop,Wine Bar,Indian Restaurant
6,Manhattan,Lincoln Square,40.773529,-73.985338,0,Plaza,Italian Restaurant,Café,Gym / Fitness Center,Theater
7,Manhattan,Chelsea,40.744035,-74.003116,0,Coffee Shop,Art Gallery,Ice Cream Shop,Café,American Restaurant
8,Manhattan,Greenwich Village,40.726933,-73.999914,0,Italian Restaurant,Café,Sushi Restaurant,Burger Joint,Spa
9,Manhattan,East Village,40.727847,-73.982226,0,Mexican Restaurant,Bar,Coffee Shop,Cocktail Bar,Speakeasy


We notice that both the clusters are actually pretty similar in the fact that at both places the most common venue is generally a food outlet but the general difeerence we can see is that the most visited plaves in toronto for cluster 0 has majority food outlets and in that to the most common one is either a Cafe or a Coffe shop. But in case of Manhattan it also has Gyms, Spas and Yoga studious within the cluster and Coffe Shops although still present are not as dominant as in case of Toronto.

After exploring the clusters we come across the observation that cluster 0 in toronto and cluster 3 in manhattan generally contain areas related to food outlets.
So, if one was to decide on where the  competition is already high or where one could find a more choice we could use this information.

In [94]:
food_ny = ny_clstr.loc[ny_clstr["Cluster Labels"]==3].reset_index(drop=True)
food_ny.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Deli / Bodega,Mobile Phone Shop,Chinese Restaurant
1,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Lounge,Bakery,Café,Restaurant
2,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Mexican Restaurant,Coffee Shop,Deli / Bodega,Café
3,Manhattan,Manhattanville,40.816934,-73.957385,3,Coffee Shop,Seafood Restaurant,Park,Bakery,Mexican Restaurant
4,Manhattan,East Harlem,40.792249,-73.944182,3,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Sandwich Place
5,Manhattan,Roosevelt Island,40.762160,-73.949168,3,Park,Deli / Bodega,Dog Run,Coffee Shop,Greek Restaurant
6,Manhattan,Morningside Heights,40.808000,-73.963896,3,Park,Coffee Shop,American Restaurant,Bookstore,Burger Joint
7,Manhattan,Battery Park City,40.711932,-74.016869,3,Park,Hotel,Memorial Site,Gym,Coffee Shop
8,Manhattan,Turtle Bay,40.752042,-73.967708,3,Italian Restaurant,Coffee Shop,Park,Café,Sushi Restaurant
9,Manhattan,Tudor City,40.746917,-73.971219,3,Café,Park,Mexican Restaurant,Pizza Place,Sushi Restaurant


We compare the size of both the clusters to get an idea about the food market at individual cities.

In [87]:
print("The number of food related spaces in Toronto is ",len(clstr0_tr))
print("The number of food related spaces in Manhattan is ",len(food_ny))

The number of food related spaces in Toronto is  27
The number of food related spaces in Manhattan is  10


So we see that Toronto has a much more tighter competition if one were to open up a store there as compared to Manhattan which only has around 10 areas where food is the main item of attraction. 